# 1. Intro.

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName(name='Basics').getOrCreate()  # name is just title to show on UI

### Read a *.json file (in the same folder of Notebook)

In [3]:
df = spark.read.json('people.json')

In [4]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
df.columns

['age', 'name']

In [6]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [7]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



### Get schema

In [8]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



### Make a custom schema

In [9]:
from pyspark.sql.types import (StructField, 
                               StructType, 
                               IntegerType, 
                               StringType)

In [10]:
# Make a custom schema
data_schema = [
    StructField('age', IntegerType(), True), # field name: age, field type: Interger, Nullable: True
    StructField('name', StringType(), True)  # filed name: name, field type: String, Nullable: True
]

In [11]:
# Make final structure
final_struct = StructType(fields=data_schema)

In [12]:
# Read a file with custom schema
df = spark.read.json('people.json', schema=final_struct)

In [13]:
# Show the custom schema
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Select a column

In [14]:
df['age']

Column<b'age'>

In [15]:
df.select('age')

DataFrame[age: int]

In [16]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [17]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [18]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

### Select a list of columns

In [19]:
df.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Add a column

In [20]:
df = df.withColumn('new_age', df['age']*3) # add new column name and its values
                                           # No inplace option as in pandas

In [21]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [22]:
df.show()

+----+-------+-------+
| age|   name|new_age|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     90|
|  19| Justin|     57|
+----+-------+-------+



### Change a column name

In [23]:
df = df.withColumnRenamed(existing='new_age', new='test') # Note: no inplace option as in pandas
df.show()

+----+-------+----+
| age|   name|test|
+----+-------+----+
|null|Michael|null|
|  30|   Andy|  90|
|  19| Justin|  57|
+----+-------+----+



### Interaction with SQL: Spark.sql

In [24]:
# Creates or replaces a local temporary view
df.createOrReplaceTempView(name='people') # lifetime is tied to the :class:`SparkSession`

In [25]:
# SQL query
results = spark.sql('SELECT * FROM people')

In [26]:
results.show()

+----+-------+----+
| age|   name|test|
+----+-------+----+
|null|Michael|null|
|  30|   Andy|  90|
|  19| Justin|  57|
+----+-------+----+



In [27]:
results_age = spark.sql('SELECT * FROM people WHERE age=19')

In [28]:
results_age.show()

+---+------+----+
|age|  name|test|
+---+------+----+
| 19|Justin|  57|
+---+------+----+



# 2. Spark dataframe basics operation

#### Read a *.csv file

In [29]:
df = spark.read.csv('appl_stock.csv', 
                    inferSchema=True, # infers the input schema automatically from data
                    header=True)

In [30]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [31]:
df.head(2)

[Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039),
 Row(Date='2010-01-05', Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002)]

#### Filter by SQL query

In [32]:
df.filter('Close < 100').show(5)  # just show 5 first rows

+----------+---------+---------+---------+---------+--------+---------+
|      Date|     Open|     High|      Low|    Close|  Volume|Adj Close|
+----------+---------+---------+---------+---------+--------+---------+
|2014-06-09|92.699997|93.879997|    91.75|93.699997|75415000|88.906324|
|2014-06-10|94.730003|95.050003|    93.57|    94.25|62777000|89.428189|
|2014-06-11|94.129997|94.760002|93.470001|93.860001|45681000|89.058142|
|2014-06-12|94.040001|94.120003|91.900002|92.290001|54749000|87.568463|
|2014-06-13|92.199997|92.440002|90.879997|91.279999|54525000|86.610132|
+----------+---------+---------+---------+---------+--------+---------+
only showing top 5 rows



In [33]:
df.filter('Close < 100').select(['Open', 'Close']).show(5)

+---------+---------+
|     Open|    Close|
+---------+---------+
|92.699997|93.699997|
|94.730003|    94.25|
|94.129997|93.860001|
|94.040001|92.290001|
|92.199997|91.279999|
+---------+---------+
only showing top 5 rows



#### Filter by dataframe operation

In [34]:
df.filter(df['Close'] < 100).show(5)

+----------+---------+---------+---------+---------+--------+---------+
|      Date|     Open|     High|      Low|    Close|  Volume|Adj Close|
+----------+---------+---------+---------+---------+--------+---------+
|2014-06-09|92.699997|93.879997|    91.75|93.699997|75415000|88.906324|
|2014-06-10|94.730003|95.050003|    93.57|    94.25|62777000|89.428189|
|2014-06-11|94.129997|94.760002|93.470001|93.860001|45681000|89.058142|
|2014-06-12|94.040001|94.120003|91.900002|92.290001|54749000|87.568463|
|2014-06-13|92.199997|92.440002|90.879997|91.279999|54525000|86.610132|
+----------+---------+---------+---------+---------+--------+---------+
only showing top 5 rows



In [35]:
df.filter(df['Close'] < 100).select(['Open', 'Close']).show(5)

+---------+---------+
|     Open|    Close|
+---------+---------+
|92.699997|93.699997|
|94.730003|    94.25|
|94.129997|93.860001|
|94.040001|92.290001|
|92.199997|91.279999|
+---------+---------+
only showing top 5 rows



#### Filter by multiple conditions using operators: & (and), | (or), ~ (not), == (equal) ...

In [36]:
df.filter((df['Close'] < 100) & (df['Open'] < 92)).show()

+----------+---------+-----------------+---------+-----------------+---------+-----------------+
|      Date|     Open|             High|      Low|            Close|   Volume|        Adj Close|
+----------+---------+-----------------+---------+-----------------+---------+-----------------+
|2014-06-16|91.510002|            92.75|91.449997|        92.199997| 35561000|        87.483064|
|2014-06-20|91.849998|        92.550003|90.900002|        90.910004|100898000|        86.259066|
|2014-06-23|    91.32|        91.620003|90.599998|90.83000200000001| 43694000|        86.183157|
|2014-06-24|    90.75|        91.739998|90.190002|        90.279999| 39036000|        85.661292|
|2014-06-25|90.209999|        90.699997|89.650002|        90.360001| 36869000|        85.737201|
|2014-06-26|90.370003|        91.050003|89.800003|        90.900002| 32629000|        86.249576|
|2014-06-27|    90.82|             92.0|90.769997|        91.980003| 64029000|        87.274325|
|2016-05-13|     90.0|91.66999

#### collect: to get output as a variable

In [37]:
results = df.filter((df['Close'] < 100) & (df['Open'] < 92)).collect()

In [38]:
results  # a list of rows

[Row(Date='2014-06-16', Open=91.510002, High=92.75, Low=91.449997, Close=92.199997, Volume=35561000, Adj Close=87.483064),
 Row(Date='2014-06-20', Open=91.849998, High=92.550003, Low=90.900002, Close=90.910004, Volume=100898000, Adj Close=86.259066),
 Row(Date='2014-06-23', Open=91.32, High=91.620003, Low=90.599998, Close=90.83000200000001, Volume=43694000, Adj Close=86.183157),
 Row(Date='2014-06-24', Open=90.75, High=91.739998, Low=90.190002, Close=90.279999, Volume=39036000, Adj Close=85.661292),
 Row(Date='2014-06-25', Open=90.209999, High=90.699997, Low=89.650002, Close=90.360001, Volume=36869000, Adj Close=85.737201),
 Row(Date='2014-06-26', Open=90.370003, High=91.050003, Low=89.800003, Close=90.900002, Volume=32629000, Adj Close=86.249576),
 Row(Date='2014-06-27', Open=90.82, High=92.0, Low=90.769997, Close=91.980003, Volume=64029000, Adj Close=87.274325),
 Row(Date='2016-05-13', Open=90.0, High=91.66999799999999, Low=90.0, Close=90.519997, Volume=44392800, Adj Close=89.1857170

#### asDict() method

In [39]:
first_row = results[0]
first_row

Row(Date='2014-06-16', Open=91.510002, High=92.75, Low=91.449997, Close=92.199997, Volume=35561000, Adj Close=87.483064)

In [40]:
first_row.asDict()

{'Date': '2014-06-16',
 'Open': 91.510002,
 'High': 92.75,
 'Low': 91.449997,
 'Close': 92.199997,
 'Volume': 35561000,
 'Adj Close': 87.483064}

In [41]:
first_row.asDict()['Volume']

35561000

# 3. Groupby and Aggregation

In [42]:
df = spark.read.csv('sales_info.csv', inferSchema=True, header=True)
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [43]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



### Groupby

In [44]:
df.groupBy('Company').mean().show() # functions: mean(), max(), min(), sum(), count(), etc

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



### Aggregation

In [45]:
df.agg({'Sales': 'sum'}).show()  # Aggregation at column "Sales" with sum() function

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



### GroupBy and Aggregation together

In [46]:
df_comp = df.groupBy('Company')

In [47]:
df_comp.agg({'Sales': 'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



### import functions from spark

In [48]:
from pyspark.sql.functions import (countDistinct, 
                                   avg, 
                                   stddev,
                                   format_number)

In [49]:
df.select(countDistinct('Company')).show()

+-----------------------+
|count(DISTINCT Company)|
+-----------------------+
|                      4|
+-----------------------+



In [50]:
df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



#### alias to change column name

In [51]:
df.select(avg('Sales').alias('Avg Sales')).show()

+-----------------+
|        Avg Sales|
+-----------------+
|360.5833333333333|
+-----------------+



#### format_number to show short real values

In [52]:
df_std = df.select(stddev('Sales').alias('std'))
df_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [53]:
df_std.select(format_number('std', 2).alias('std')).show()

+------+
|   std|
+------+
|250.09|
+------+



#### orderBy

In [54]:
df.orderBy('Sales', ascending=False).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



# 4. Missing data

In [55]:
df = spark.read.csv('ContainsNull.csv', inferSchema=True, header=True)
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### drop: how option

In [56]:
# drop 
df.na.drop(how='any').show() # how='any': any null values
                             # how='all': only drop if all null values in a column

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



### drop: threshold option

In [57]:
# drop null values if in one row they ocurr in 2 columns (threshol=2)
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### drop: subset option

In [58]:
df.na.drop(subset=['Sales']).show()  # drop null values in only a subset columns

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### fill

In [59]:
df.na.fill('No name', subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No name| null|
|emp3|No name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [60]:
# Fill with mean value
from pyspark.sql.functions import mean
mean_sales = df.select(mean(df['Sales'])).collect()
print(f'mean_sales = {mean_sales}')  # [Row(avg(Sales)=400.5)]

df.na.fill(mean_sales[0][0], subset=['Sales']).show()

mean_sales = [Row(avg(Sales)=400.5)]
+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



# 5. Date and Timestamp

In [61]:
df = spark.read.csv('appl_stock.csv', inferSchema=True, header=True)
df.show(3)

+----------+----------+----------+------------------+----------+---------+------------------+
|      Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|
+----------+----------+----------+------------------+----------+---------+------------------+
only showing top 3 rows



In [62]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [63]:
from pyspark.sql.functions import (dayofmonth,
                                   dayofweek,
                                   dayofyear,
                                   weekofyear,
                                   hour,
                                   month,
                                   year,
                                   format_number,
                                   date_format)

In [64]:
# Make a "Year" column
new_df = df.withColumn('Year', year(df['Date']))
new_df.show(3)

+----------+----------+----------+------------------+----------+---------+------------------+----+
|      Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|Year|
+----------+----------+----------+------------------+----------+---------+------------------+----+
|2010-01-04|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|2010|
|2010-01-05|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|2010|
|2010-01-06|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|2010|
+----------+----------+----------+------------------+----------+---------+------------------+----+
only showing top 3 rows



In [65]:
# groupBy Year and calculate mean
new_df.groupBy('Year').mean().show(3)

+----+------------------+------------------+------------------+------------------+-------------------+------------------+---------+
|Year|         avg(Open)|         avg(High)|          avg(Low)|        avg(Close)|        avg(Volume)|    avg(Adj Close)|avg(Year)|
+----+------------------+------------------+------------------+------------------+-------------------+------------------+---------+
|2015|120.17575393253965|121.24452385714291| 118.8630954325397|120.03999980555547| 5.18378869047619E7|115.96740080555561|   2015.0|
|2013| 473.1281355634922| 477.6389272301587|468.24710264682557| 472.6348802857143|         1.016087E8| 62.61798788492063|   2013.0|
|2014| 295.1426195357143|297.56103184523823| 292.9949599801587| 295.4023416507935|6.315273055555555E7| 87.63583323809523|   2014.0|
+----+------------------+------------------+------------------+------------------+-------------------+------------------+---------+
only showing top 3 rows



In [66]:
# groupBy Year and calculate mean then select only 2 columns
result = new_df.groupBy('Year').mean().select(['Year', 'avg(Close)'])
result.show()

+----+------------------+
|Year|        avg(Close)|
+----+------------------+
|2015|120.03999980555547|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2012| 576.0497195640002|
|2016|104.60400786904763|
|2010| 259.8424600000002|
|2011|364.00432532142867|
+----+------------------+



In [67]:
result = result.withColumnRenamed(existing='avg(Close)', new='Avg Closing Price')
result.show()

+----+------------------+
|Year| Avg Closing Price|
+----+------------------+
|2015|120.03999980555547|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2012| 576.0497195640002|
|2016|104.60400786904763|
|2010| 259.8424600000002|
|2011|364.00432532142867|
+----+------------------+



In [68]:
result = result.select(['Year', format_number('Avg Closing Price', 2).alias('Avg Closing')])
result.show()

+----+-----------+
|Year|Avg Closing|
+----+-----------+
|2015|     120.04|
|2013|     472.63|
|2014|     295.40|
|2012|     576.05|
|2016|     104.60|
|2010|     259.84|
|2011|     364.00|
+----+-----------+

